In [1]:
import itk
import dask.array as da
import dask
from dask.array.image import imread
from dask.distributed import Client, progress, LocalCluster
import dask.array as da
client = Client(n_workers=8)#, serializers=['pickle'])#, processes=False)
client

Client Scheduler: tcp://127.0.0.1:43285 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.63 GB


In [2]:
def load_image(path):
    itk_img = itk.imread(path)
    image_np = itk.array_from_image(itk_img)
    return image_np 
np_img = load_image("data/test_data.tif")

In [3]:
def denoise(block):
    import itk
    import copy
    import numpy as np
    x = itk.median_image_filter(block)
    
    return x#np.asarray(x)

In [4]:
type(denoise(np_img))

itk.itkPyBufferPython.NDArrayITKBase

In [5]:
type(np_img)

numpy.ndarray

In [6]:
# template_type = type(denoise(np_img))
# from distributed.protocol import register_generic, dask_serialize, dask_deserialize
# from typing import Any, Dict, Tuple, List, Union
            
# @dask_deserialize.register(template_type)
# def deserialize(header: Dict, frames: List[bytes]) -> template_type:
# #     import pdb; pdb.set_trace()
#     typ = type(np_img)
#     loads = dask_deserialize.dispatch(typ)
#     return template_type(loads(header, frames))

In [6]:
chunked_data  = da.from_array(np_img, chunks=(200, 200, 5))  ## 40 blocks
scattered_data = client.scatter(chunked_data, broadcast=True)

In [7]:
scattered_data.result()

,Array,Chunk
Bytes,5.24 MB,20.00 kB
Shape,"(20, 512, 512)","(20, 200, 5)"
Count,310 Tasks,309 Chunks
Type,uint8,numpy.ndarray


In [8]:
denoised = client.submit(denoise, scattered_data).result()
type(denoised)

itk.itkPyBufferPython.NDArrayITKBase